In [5]:
# !pip install boto3 opensearch-py python-dotenv unstructured pdf2image pillow streamlit

In [133]:
import os
import boto3
from dotenv import load_dotenv
from requests_aws4auth import AWS4Auth
from opensearchpy import OpenSearch, RequestsHttpConnection

load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")

# AWS 자격 증명 및 서명 설정
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
credentials = session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    aws_region,
    'aoss' # Amazon OpenSearch Serverless 서비스명
)

s3_client = session.client('s3')
bedrock_client = session.client('bedrock-runtime')
s3_bucket_name = os.getenv("S3_BUCKET_NAME")

# OpenSearch Serverless 클라이언트 설정
opensearch_host = os.getenv("OPENSEARCH_COLLECTION_HOST")
opensearch_index_name = os.getenv("OPENSEARCH_TEST_INDEX_NAME")
opensearch_client = OpenSearch(
    hosts=[{'host': opensearch_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=30
)

In [67]:
import os

def upload_pdf_to_s3(file_path):
    """
    로컬 PDF 파일을 S3 버킷에 업로드합니다.
    """
    file_name = os.path.basename(file_path)
    try:
        s3_client.upload_file(file_path, s3_bucket_name, file_name)
        print(f"'{file_name}' 파일이 S3 버킷 '{s3_bucket_name}'에 성공적으로 업로드되었습니다.")
        return file_name
    except Exception as e:
        print(f"S3 업로드 중 오류 발생: {e}")
        return None

In [68]:
# 업로드할 PDF 파일 경로 지정 (예시)
local_pdf_path = "pdf-files/accu.pdf"
s3_file_name = upload_pdf_to_s3(local_pdf_path)

'accu.pdf' 파일이 S3 버킷 'mjkwon-s3'에 성공적으로 업로드되었습니다.


In [7]:
# !pip install --upgrade unstructured unstructured-inference opensearch-py
# !pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
# !pip install onnxruntime

In [84]:
# opensearch_index_name = os.getenv("OPENSEARCH_TEST_INDEX_NAME")

In [85]:
# process_pdf_and_index_opensearch 함수를 나누어 실행하는 첫 번째 셀

import json
import boto3
from opensearchpy.helpers import bulk
from opensearchpy.exceptions import NotFoundError
from unstructured.partition.pdf import partition_pdf
from io import BytesIO
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

# s3_file_name = "accu.pdf"  # 예시 파일명

# unstructured 파편화 문제 해결
# 가장 근본적인 원인은 unstructured 라이브러리가 PDF의 레이아웃(특히 이미지 내 텍스트나 복잡한 표)을 제대로 인식하지 못하고 파편화된 텍스트를 생성했기 때문입니다. 이 문제는 단순한 텍스트 검색(keyword search)으로는 해결하기 어렵습니다.

# 2. 해결 방법: 벡터 검색(k-NN) 도입
# 텍스트 파편화 문제를 해결하고 질문의 의미를 정확하게 파악하기 위해서는 **벡터 검색(k-NN)**을 도입하는 것이 가장 효과적입니다. 벡터 검색은 텍스트의 의미를 숫자로 표현한 벡터를 생성하고, 질문 벡터와 가장 유사한 문서 벡터를 찾아냅니다.

# try:
#     print("1. S3에서 PDF 파일 다운로드 중...")
#     s3_object = s3_client.get_object(Bucket=s3_bucket_name, Key=s3_file_name)
#     pdf_content = s3_object['Body'].read()
#     print("다운로드 완료.")

#     print("2. unstructured를 사용하여 PDF 내용 분할 중...")
#     elements = partition_pdf(
#         file=BytesIO(pdf_content),
#         strategy="hi_res",
#         infer_table_structure=False,
#         languages=["kor", "eng"],
#         extract_images_in_pdf=True,
#         image_output_dir_path="./images"
#     )
#     print(f"unstructured 분할 완료. 추출된 요소 수: {len(elements)}")

#     print("---")
#     print("첫 번째 단계 성공. 다음 셀로 이동하세요.")
    
# except Exception as e:
#     print(f"**오류 발생 (1단계): {e}**")


# 필요한 작업:
# 임베딩 모델 선택: Bedrock의 titan-embed-text-v1과 같은 텍스트 임베딩 모델을 사용합니다.
# 임베딩 벡터 생성: PDF에서 추출한 각 텍스트 청크(chunk)를 임베딩 모델에 넣어 벡터로 변환합니다.
# OpenSearch에 벡터 저장: 텍스트와 함께 벡터를 OpenSearch 인덱스에 저장합니다.
# 검색 쿼리 수정: 사용자 질문을 벡터로 변환한 후, OpenSearch의 knn 쿼리를 사용해 유사한 벡터를 가진 문서를 검색합니다.

# 텍스트 청킹(Chunking) 전략: unstructured가 추출한 텍스트를 문장이나 단락 단위로 묶는 추가적인 전처리 과정을 거쳐 임베딩의 품질을 높입니다.

def process_pdf_and_index_opensearch_with_embeddings(s3_file_name):

    try:
            
            # S3에서 PDF 다운로드
            s3_object = s3_client.get_object(Bucket=s3_bucket_name, Key=s3_file_name)
            pdf_content = s3_object['Body'].read()

            # unstructured로 PDF 내용 분할
            elements = partition_pdf(
                file=BytesIO(pdf_content),
                strategy="hi_res",
                infer_table_structure=False,
                languages=["kor", "eng"],
                extract_images_in_pdf=True,
                image_output_dir_path="./figures"
            )

            # 텍스트 청킹을 위한 전처리
            text_content = "\n".join([el.text for el in elements if el.text])
            ## 청크 크기 조절 필요
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
            chunks = text_splitter.split_text(text_content)

            # 인덱스 존재 여부를 확인하는 새로운 방식
            index_exists = True
            try:
                opensearch_client.indices.get(index=opensearch_index_name)
            except NotFoundError:
                index_exists = False

            if index_exists:
                print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 이미 존재합니다.")
                opensearch_client.indices.delete(index=opensearch_index_name)
                print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 삭제 완료")
            # else:
            #     print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 존재하지 않습니다. 새로 생성합니다.")
            #     opensearch_client.indices.create(index=opensearch_index_name)
            #     print(f"OpenSearch 인덱스 '{opensearch_index_name}' 생성 완료.")
            
            # print("---")

            else:
                print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 존재하지 않습니다. 새로 생성합니다.")
                mapping = {
                    "settings": {
                        "index.knn": True,
                    },
                    "mappings": {
                        "properties": {
                            "text": {"type": "text"},
                            "embedding": {
                                "type": "knn_vector",
                                "dimension": 1024, # Cohere 모델 차원 수
                                "method": {
                                    "name": "hnsw",
                                    "engine": "nmslib"
                                }
                            }
                        }
                    }
                }
            opensearch_client.indices.create(index=opensearch_index_name, body=mapping)
            print(f"OpenSearch 인덱스 '{opensearch_index_name}' 생성 완료.")

            
            # OpenSearch 인덱스 존재 여부 확인 및 생성 (에러 발생: OpenSearch 인덱싱 중 오류 발생: IndicesClient.exists() takes 1 positional argument but 2 positional arguments (and 2 keyword-only arguments) were given
            # if not opensearch_client.indices.exists(opensearch_index_name):
            #     # 벡터 검색을 위한 매핑 정보 포함
            #     mapping = {
            #         "settings": {
            #             "index.knn": True
            #         },
            #         "mappings": {
            #             "properties": {
            #                 "text": {"type": "text"},
            #                 "embedding": {
            #                     "type": "knn_vector",
            #                     "dimension": 1536, # Titan Embed Text 모델 차원 수
            #                     "method": {
            #                         "name": "hnsw",
            #                         "engine": "nmslib"
            #                     }
            #                 }
            #             }
            #         }
            #     }
            #     opensearch_client.indices.create(index=opensearch_index_name, body=mapping)
            #     print(f"OpenSearch 인덱스 '{opensearch_index_name}' 생성 완료.")

            docs = []
            # for i, el in enumerate(elements):
            #     text = el.text if el.text else ""
            #     if text:
            #         # Bedrock Titan Embeddings 모델 호출
            #         bedrock_model_id = 'amazon.titan-embed-text-v1'
            #         response = bedrock_client.invoke_model(
            #             modelId=bedrock_model_id,
            #             body=json.dumps({"inputText": text})
            #         )
            #         embedding = json.loads(response['body'].read())['embedding']

            #         doc = {
            #             'text': text,
            #             'source': s3_file_name,
            #             'page_number': el.metadata.page_number if hasattr(el.metadata, 'page_number') else None,
            #             'embedding': embedding
            #         }
            #         docs.append({
            #             '_index': opensearch_index_name,
            #             '_source': doc
            #         })

            for i, chunk in enumerate(chunks):
                bedrock_model_id = 'cohere.embed-multilingual-v3'
                response = bedrock_client.invoke_model(
                    modelId=bedrock_model_id,
                    body=json.dumps({"texts": [chunk], "input_type": "search_document"})
                )
                embedding = json.loads(response['body'].read())['embeddings'][0]

                doc = {
                'text': chunk,
                'source': s3_file_name,
                'page_number': i + 1,
                'embedding': embedding
            }
            docs.append({
                '_index': opensearch_index_name,
                '_source': doc
            })

            bulk(opensearch_client, docs)
            print(f"PDF 내용이 OpenSearch에 성공적으로 인덱싱되었습니다. 문서 수: {len(docs)}")

    except Exception as e:
        print(f"OpenSearch 인덱싱 중 오류 발생: {e}")




In [86]:
# 함수 호출
process_pdf_and_index_opensearch_with_embeddings(s3_file_name)

short text: "Acculnsight+ 2.0 #e2|At WS". Defaulting to English.
short text: "2021. 09 DataSet". Defaulting to English.
short text: "i SK SIA} C&C". Defaulting to English.
short text: "01 ZRAE Fel". Defaulting to English.
short text: "PIPELINE". Defaulting to English.
short text: "/Q |". Defaulting to English.
short text: "demo-for-mosiler". Defaulting to English.
short text: ";". Defaulting to English.
short text: "=". Defaulting to English.
short text: "test22". Defaulting to English.
short text: "AAS". Defaulting to English.
short text: "0". Defaulting to English.
short text: ";". Defaulting to English.
short text: "| 01". Defaulting to English.
short text: "[HO OS o [He". Defaulting to English.
short text: "Jlo". Defaulting to English.
short text: "|". Defaulting to English.
short text: "Jlo". Defaulting to English.
short text: "PIPELINE". Defaulting to English.
short text: "01 BAAR tel". Defaulting to English.
short text: "PIPELINE". Defaulting to English.
short text: "03 HABeFP. 

OpenSearch 인덱스 'test-index'이(가) 존재하지 않습니다. 새로 생성합니다.
OpenSearch 인덱스 'test-index' 생성 완료.
PDF 내용이 OpenSearch에 성공적으로 인덱싱되었습니다. 문서 수: 1


In [87]:
def get_rag_answer_from_bedrock_with_embeddings(query):
    try:
        # 1. 사용자 질문을 임베딩 벡터로 변환
        # bedrock_model_id = 'amazon.titan-embed-text-v1'
        # response = bedrock_client.invoke_model(
        #     modelId=bedrock_model_id,
        #     body=json.dumps({"inputText": query})
        # )
        # query_embedding = json.loads(response['body'].read())['embedding']

        # chunking, 사용자 질문을 임베딩 벡터로 변환
        bedrock_model_id_embedding = 'cohere.embed-multilingual-v3'
        print(f"임베딩 모델: {bedrock_model_id_embedding}")

        response = bedrock_client.invoke_model(
            modelId=bedrock_model_id_embedding,
            body=json.dumps({"texts": [query], "input_type": "search_query"})
        )
        query_embedding = json.loads(response['body'].read())['embeddings'][0]
        
        # 2. OpenSearch에서 벡터 검색 (k-NN)
        # search_query = {
        #     "size": 5,
        #     "query": {
        #         "knn": {
        #             "embedding": {
        #                 "vector": query_embedding,
        #                 "k": 5
        #             }
        #         }
        #     },
        #     "_source": ["text"] # 텍스트 필드만 가져와서 컨텍스트로 사용
        # }

        # 2. OpenSearch에서 하이브리드 검색 (knn + multi_match)
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "should": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["text^2"],
                                "type": "best_fields"
                            }
                        },
                        {
                            "knn": {
                                "embedding": {
                                    "vector": query_embedding,
                                    "k": 5
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1 # 하나라도 일치하면 문서 반환
                }
            },
            "_source": ["text"]
        }

        # print(f"\nOpenSearch 검색 쿼리:\n{json.dumps(search_query, indent=2, ensure_ascii=False)}")

        response = opensearch_client.search(index=opensearch_index_name, body=search_query)
        hits = response['hits']['hits']
        context_docs = [hit['_source']['text'] for hit in hits]
        
        print(f"\n검색된 문서 수: {len(hits)}개")
        for i, doc in enumerate(context_docs):
            print(f"--- 검색된 문서 #{i+1} ---\n{doc}\n------------------------")
        
        if not context_docs:
            print("관련 문서를 찾지 못했습니다. Bedrock에 전달할 컨텍스트가 없습니다.")
            return "죄송합니다. 질문에 대한 관련 문서를 찾을 수 없습니다."

        context = "\n\n".join(context_docs)
        
        # 3. Bedrock LLM에 프롬프트 전달
        llm_model_id = os.getenv("BEDROCK_MODEL_ID")
        print(f"\nBedrock 모델: {llm_model_id}")

        if 'claude' in llm_model_id:
            prompt = f"""
            다음은 사용자 질문에 답변하기 위한 참고 자료입니다.
            <자료>
            {context}
            </자료>
            당신은 유용한 AI 비서입니다. 제공된 자료만을 바탕으로 사용자의 질문에 한국어로 상세하고 친절하게 답변해주세요. 만약 자료에 답변이 없다면, "죄송합니다. 제공된 문서에는 답변이 없습니다."라고 말해주세요. 절대 자료에 없는 정보를 임의로 생성하지 마세요.
            사용자 질문: {query}
            답변:
            """
            body = json.dumps({"anthropic_version": "bedrock-2023-05-31", "max_tokens": 1000, "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}], "temperature": 0.5})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body, contentType="application/json", accept="application/json")
            llm_answer = json.loads(response_body.get('body').read())['content'][0]['text']

        else:
            prompt = f"""
            You are a helpful AI assistant. Use only the provided context to answer the user's question in Korean. If the answer is not in the context, say "죄송합니다. 제공된 문서에는 답변이 없습니다." Do not make up information.
            Context:
            {context}
            Question: {query}
            Answer:
            """
            body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 1000, "stopSequences": [], "temperature": 0.5, "topP": 0.9}})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body)
            llm_answer = json.loads(response_body.get('body').read())['results'][0]['outputText']

        print(f"\n최종 답변: {llm_answer}")
        return llm_answer

    except Exception as e:
        print(f"\n오류 발생: {e}")
        return f"RAG 처리 중 오류가 발생했습니다: {e}"

In [113]:
opensearch_index_name = os.getenv("OPENSEARCH_TEST_INDEX_NAME")

In [116]:
# PDF 내용과 관련된 질문으로 테스트
# 프로젝트 관리: 프로젝트 관리 페이지에서 어떤 작업을 할 수 있습니까?
# 워크플로우 관리: 워크플로우를 삭제하려면 어떤 버튼을 눌러야 합니까? 
# 사용자 권한: user001의 워크플로우 권한은 무엇입니까? 
# 사용자 이관: 사용자의 탈퇴 또는 계정 삭제 시 사용자를 이관하는 기능은 어떤 계정만 가능합니까? 

query = "사용자의 탈퇴 또는 계정 삭제 시 사용자를 이관하는 기능은 어떤 계정만 가능합니까?" # PDF 내용에 따라 질문을 변경하세요.

# 다른 일반 사용자 계정인 user001, user002 등은 사용자 이관 기능을 사용할 수 없습니다.
# 따라서 사용자 탈퇴나 삭제 시 사용자 이관 기능을 사용할 수 있는 계정은 sso_admin 뿐이라고 답변드리겠습니다. 제공된 자료 내용에 근거한 답변입니다.

# query = "프로젝트 관리에서 특정 프로젝트 수정을 하는 방법은? 순서대로 알려주세요."

answer = get_rag_answer_from_bedrock_with_embeddings(query)

print(f"질문: {query}")
print(f"답변: {answer}")

임베딩 모델: cohere.embed-multilingual-v3


INFO:opensearch:POST https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-index/_search [status:200 request:0.174s]



검색된 문서 수: 1개
--- 검색된 문서 #1 ---
221
15
03 GAY oOlOlA| ete]
@ AHA 00/4) Be
A O/O|A] Sf HY O|O|A| SAT B= AS. workspace WS SH OfO|A] A|A LOA, Of O]A| SLE MES. FE OOAAY Al “O|O]A| MYA" SS +B SESNSA oO.
© #4 / = ofgjz) wey
Deployment 412, workspace= BAY O/O|A\2 FH B.S Ss olla MHS.
Xt 2 oO
Aus Aa re
MODELER
End of Document
------------------------

Bedrock 모델: anthropic.claude-v2

최종 답변: 제공된 자료를 보면 프로젝트 관리에서 특정 프로젝트를 수정하는 방법에 대한 정보가 포함되어 있지 않습니다. 따라서 정확한 답변을 드리기 어렵습니다. 죄송합니다만, 제공된 문서에는 이에 대한 답변이 없습니다. 프로젝트 수정 방법에 대한 질문이시라면, 보다 자세한 정보를 제공해 주시면 감사하겠습니다.
질문: 프로젝트 관리에서 특정 프로젝트 수정을 하는 방법은? 순서대로 알려주세요.
답변: 제공된 자료를 보면 프로젝트 관리에서 특정 프로젝트를 수정하는 방법에 대한 정보가 포함되어 있지 않습니다. 따라서 정확한 답변을 드리기 어렵습니다. 죄송합니다만, 제공된 문서에는 이에 대한 답변이 없습니다. 프로젝트 수정 방법에 대한 질문이시라면, 보다 자세한 정보를 제공해 주시면 감사하겠습니다.


In [121]:
opensearch_index_name = os.getenv("OPENSEARCH_TEST_INDEX_NAME")

In [134]:
import logging
# 로깅 설정
logging.basicConfig(level=logging.INFO)

In [138]:
# ========================
# 2. 인덱싱 함수 (텍스트 + 이미지)
# ========================
def process_pdf_and_index_opensearch_with_images(s3_file_name):
    try:
        logging.info("S3에서 PDF 파일 다운로드 중...")
        s3_object = s3_client.get_object(Bucket=s3_bucket_name, Key=s3_file_name)
        pdf_content = s3_object['Body'].read()
        logging.info("다운로드 완료.")

        logging.info("unstructured를 사용하여 PDF 내용 분할 중...")
        elements = partition_pdf(
            file=BytesIO(pdf_content),
            strategy="hi_res",
            infer_table_structure=False,
            languages=["kor", "eng"],
            extract_images_in_pdf=True,
            image_output_dir_path="./images"
        )
        logging.info(f"unstructured 분할 완료. 추출된 요소 수: {len(elements)}")

        pages = {}
        for el in elements:
            page_num = el.metadata.page_number if hasattr(el.metadata, 'page_number') else None
            if page_num not in pages:
                pages[page_num] = {'text': [], 'images': []}
            
            if el.text:
                pages[page_num]['text'].append(el.text)
            
            if hasattr(el.metadata, 'image_path') and el.metadata.image_path:
                pages[page_num]['images'].append(os.path.basename(el.metadata.image_path))

            
        # OpenSearch 인덱스 존재 여부 확인 및 생성
        index_exists = True
        try:
            opensearch_client.indices.get(index=opensearch_index_name)
        except NotFoundError:
            index_exists = False

        if index_exists:
            print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 이미 존재합니다.")
            opensearch_client.indices.delete(index=opensearch_index_name)
            print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 삭제 완료")
        else:
            print(f"OpenSearch 인덱스 '{opensearch_index_name}'이(가) 존재하지 않습니다. 새로 생성합니다.")
        
        mapping = {
            "settings": {
                "index.knn": True,
            },
            "mappings": {
                "properties": {
                    "text": {"type": "text"},
                    "embedding": {
                        "type": "knn_vector",
                        "dimension": 1024,
                        "method": {
                            "name": "hnsw",
                            "engine": "nmslib"
                        }
                    },
                    "image_paths": {"type": "keyword"},
                    "page_number": {"type": "long"}
                }
            }
        }
                
        opensearch_client.indices.create(index=opensearch_index_name, body=mapping)
        logging.info(f"OpenSearch 인덱스 '{opensearch_index_name}' 생성 완료.")

        docs = []
        for page_num, content in pages.items():
            if not content['text'] and not content['images']:
                continue
            combined_text = "\n".join(content['text'])
            
            bedrock_model_id = 'cohere.embed-multilingual-v3'
            response = bedrock_client.invoke_model(
                modelId=bedrock_model_id,
                body=json.dumps({"texts": [combined_text], "input_type": "search_document"})
            )
            embedding = json.loads(response['body'].read())['embeddings'][0]

            doc = {
                'text': combined_text,
                'source': s3_file_name,
                'page_number': page_num,
                'embedding': embedding,
                'image_paths': content['images']
            }
            docs.append({
                '_index': opensearch_index_name,
                '_source': doc
            })

        bulk(opensearch_client, docs)
        logging.info(f"PDF 내용 및 이미지 정보가 OpenSearch에 성공적으로 인덱싱되었습니다. 문서 수: {len(docs)}")

    except Exception as e:
        logging.error(f"오류 발생: {e}")

In [139]:
# ========================
# 3. RAG 함수 (하이브리드 검색 + 이미지 출력)
# ========================
def get_rag_answer_from_bedrock_with_images(query):
    try:
        bedrock_model_id_embedding = 'cohere.embed-multilingual-v3'
        response = bedrock_client.invoke_model(
            modelId=bedrock_model_id_embedding,
            body=json.dumps({"texts": [query], "input_type": "search_query"})
        )
        query_embedding = json.loads(response['body'].read())['embeddings'][0]
        
        search_query = {
            "size": 3,
            "query": {
                "bool": {
                    "should": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["text^2"]
                            }
                        },
                        {
                            "knn": {
                                "embedding": {
                                    "vector": query_embedding,
                                    "k": 3
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            },
            "_source": ["text", "image_paths"]
        }
        
        response = opensearch_client.search(index=opensearch_index_name, body=search_query)
        hits = response['hits']['hits']
        context_docs = []
        image_paths = []
        
        for hit in hits:
            context_docs.append(hit['_source']['text'])
            if 'image_paths' in hit['_source'] and hit['_source']['image_paths']:
                image_paths.extend(hit['_source']['image_paths'])
        
        if not context_docs:
            return "죄송합니다. 질문에 대한 관련 문서를 찾을 수 없습니다.", []

        context = "\n\n".join(context_docs)
        
        llm_model_id = os.getenv("BEDROCK_MODEL_ID")
        
        if 'claude' in llm_model_id:
            prompt = f"""
            다음은 사용자 질문에 답변하기 위한 참고 자료입니다.
            <자료>
            {context}
            </자료>
            당신은 유용한 AI 비서입니다. 제공된 자료만을 바탕으로 사용자의 질문에 한국어로 상세하고 친절하게 답변해주세요. 만약 자료에 답변이 없다면, "죄송합니다. 제공된 문서에는 답변이 없습니다."라고 말해주세요. 절대 자료에 없는 정보를 임의로 생성하지 마세요.
            사용자 질문: {query}
            답변:
            """
            body = json.dumps({"anthropic_version": "bedrock-2023-05-31", "max_tokens": 1000, "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}], "temperature": 0.5})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body, contentType="application/json", accept="application/json")
            llm_answer = json.loads(response_body.get('body').read())['content'][0]['text']
        else:
            prompt = f"""
            You are a helpful AI assistant. Use only the provided context to answer the user's question in Korean. If the answer is not in the context, say "죄송합니다. 제공된 문서에는 답변이 없습니다." Do not make up information.
            Context:
            {context}
            Question: {query}
            Answer:
            """
            body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 1000, "stopSequences": [], "temperature": 0.5, "topP": 0.9}})
            response_body = bedrock_client.invoke_model(modelId=llm_model_id, body=body)
            llm_answer = json.loads(response_body.get('body').read())['results'][0]['outputText']

        unique_image_paths = list(set(image_paths))
        return llm_answer, unique_image_paths

    except Exception as e:
        return f"RAG 처리 중 오류가 발생했습니다: {e}", []

In [140]:
# 함수 호출
process_pdf_and_index_opensearch_with_images(s3_file_name)

INFO:root:S3에서 PDF 파일 다운로드 중...
INFO:root:다운로드 완료.
INFO:root:unstructured를 사용하여 PDF 내용 분할 중...
INFO:unstructured:PDF text extraction failed, skip text extraction...
INFO:unstructured_inference:Reading PDF for file: /var/folders/bn/5hyhxy6d0293c60lhm7g3dc40000gn/T/tmpk_7qzxxl/document.pdf ...
INFO:root:unstructured 분할 완료. 추출된 요소 수: 109


OpenSearch 인덱스 'test-index'이(가) 존재하지 않습니다. 새로 생성합니다.


INFO:opensearch:PUT https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-index [status:200 request:0.350s]
INFO:root:OpenSearch 인덱스 'test-index' 생성 완료.
INFO:opensearch:POST https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/_bulk [status:200 request:11.522s]
INFO:root:PDF 내용 및 이미지 정보가 OpenSearch에 성공적으로 인덱싱되었습니다. 문서 수: 18


In [119]:
opensearch_index_name = os.getenv("OPENSEARCH_TEST_INDEX_NAME")

In [141]:
# PDF 내용과 관련된 질문으로 테스트
# 프로젝트 관리: 프로젝트 관리 페이지에서 어떤 작업을 할 수 있습니까?
# 워크플로우 관리: 워크플로우를 삭제하려면 어떤 버튼을 눌러야 합니까? 
# 사용자 권한: user001의 워크플로우 권한은 무엇입니까? 
# 사용자 이관: 사용자의 탈퇴 또는 계정 삭제 시 사용자를 이관하는 기능은 어떤 계정만 가능합니까? 

# query = "사용자의 탈퇴 또는 계정 삭제 시 사용자를 이관하는 기능은 어떤 계정만 가능합니까?" # PDF 내용에 따라 질문을 변경하세요.

# 다른 일반 사용자 계정인 user001, user002 등은 사용자 이관 기능을 사용할 수 없습니다.
# 따라서 사용자 탈퇴나 삭제 시 사용자 이관 기능을 사용할 수 있는 계정은 sso_admin 뿐이라고 답변드리겠습니다. 제공된 자료 내용에 근거한 답변입니다.

query = "프로젝트 관리에서 특정 프로젝트 수정을 하는 방법은? 순서대로 알려주세요."

answer = get_rag_answer_from_bedrock_with_images(query)

print(f"질문: {query}")
print(f"답변: {answer}")

INFO:opensearch:POST https://puwzk1mbr6lqzo7x97d4.us-west-2.aoss.amazonaws.com:443/test-index/_search [status:200 request:0.255s]


질문: 프로젝트 관리에서 특정 프로젝트 수정을 하는 방법은? 순서대로 알려주세요.
답변: ('제공된 자료에 따르면, 프로젝트 관리에서 특정 프로젝트를 수정하는 방법은 다음과 같습니다:\n\n1. 프로젝트 관리 페이지로 이동합니다.\n\n2. 수정하려는 프로젝트를 선택합니다. \n\n3. 프로젝트 정보 옆의 편집 아이콘을 클릭합니다.\n\n4. 프로젝트 정보 수정 페이지에서 프로젝트 이름, 설명, 사용자 등을 수정합니다.\n\n5. 저장 버튼을 클릭하여 수정된 내용을 저장합니다.\n\n이와 같은 절차를 통해 프로젝트 관리에서 특정 프로젝트를 수정할 수 있습니다. 제공된 자료에서 확인할 수 있듯이 프로젝트를 선택하고 편집 아이콘을 클릭하여 수정 페이지로 이동한 후 정보를 수정하고 저장하면 됩니다.', ['figure-9-5.jpg'])


In [142]:
# !pip install streamlit_pdf_viewer

  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.8 MB/s  0:00:00
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6━━━━━━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [streamlit_pdf_viewer]eamlit_pdf_viewer]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
llama-index-core 0.13.2 requires nltk>3.8.1, but you have nltk 3.8.1 which is incompatible.
